In [ ]:
import csv
import random
import pandas as pd
import os
import datetime

USER_CREDENTIALS_FILE = "/content/user_credentials.csv"
BOOKING_TABLE_FILE = "/content/book_table.csv"


def read_user_credentials_from_csv():
    """Reads the user credentials from the user_credentials.csv file."""
    user_credentials = []
    try:
        with open(USER_CREDENTIALS_FILE, "r", encoding="utf-8-sig") as file:
            reader = csv.DictReader(file)
            for row in reader:
                user_credentials.append(row)
    except FileNotFoundError:
        pass
    return user_credentials


def write_user_credentials_to_csv(user_credentials):
    """Writes the user credentials to the user_credentials.csv file."""
    with open(USER_CREDENTIALS_FILE, "w", newline="", encoding="utf-8-sig") as file:
        fieldnames = ["Username", "Password"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(user_credentials)


def register_user(user_credentials):
    """Registers a new user and adds their credentials to the user_credentials.csv file."""
    while True:
        username = input("Enter a username: ")
        if is_username_taken(user_credentials, username):
            print("Username is already taken. Please choose a different username.")
        else:
            password = input("Enter a password: ")
            if not is_valid_password(password):
                print("Invalid password. Password must have at least 1 capital letter, 1 digit, and length more than 4.")
            else:
                user_credentials.append({"Username": username, "Password": password})
                write_user_credentials_to_csv(user_credentials)
                print("\n", "### Registration successful👏 You are now ready to login! ###", "\n")
                break


def is_username_taken(user_credentials, username):
    """Checks if a username is already taken."""
    try:
        if any(user["Username"] == username for user in user_credentials):
            return True
    except KeyError:
        pass
    return False


def is_valid_password(password):
    """Checks if a password meets the criteria: at least 1 capital letter, 1 digit, and length more than 4."""
    if not any(char.isupper() for char in password):
        return False
    if not any(char.isdigit() for char in password):
        return False
    if len(password) <= 4:
        return False
    return True


def login(user_credentials):
    """Logs in a user by verifying their credentials."""
    username = input("Enter your username: ")
    password = input("Enter your password: ")
    for user in user_credentials:
        if user["Username"] == username and user["Password"] == password:
            print("\n", "### Login successful👏 ###","\n")
            return user
    return None


def read_restaurants_from_csv():
    """Reads the restaurant details from the restaurants.csv file."""
    restaurants = []
    with open("/content/book_table-4.csv", "r", encoding="utf-8-sig") as file:
        reader = csv.DictReader(file)
        for row in reader:
            restaurants.append(row)
    return restaurants


def select_restaurant(restaurants):
    """Displays the list of restaurants and allows the user to select one."""
    print("Select a restaurant:")
    for i, restaurant in enumerate(restaurants, start=1):
        print(f"{i}. {restaurant['name']}")
    choice = input("Enter the number of the restaurant: ")
    try:
        index = int(choice) - 1
        if 0 <= index < len(restaurants):
            return restaurants[index]
        else:
            print("*** Invalid choice. Please enter a valid number. ***")
            return select_restaurant(restaurants)
    except ValueError:
        print("*** Invalid choice. Please enter a valid number. ***")
        return select_restaurant(restaurants)


def is_restaurant_in_high_demand(restaurant, date):
    """Checks if a restaurant is in high demand based on a random condition."""
    date_obj = datetime.datetime.strptime(date, "%Y-%m-%d")
    is_weekend = date_obj.weekday() >= 5
    return is_weekend or random.random() < 0.3


def create_reservation(restaurant, date, time, name, number_of_people, status):
    """Creates a new reservation."""
    reservation_id = random.randint(1000, 8000)
    reservations = read_reservations_from_csv()
    if check_reservation_availability(reservations, reservation_id, restaurant, date, time, int(number_of_people)):
        reservation = {
            "Reservation ID": reservation_id,
            "Restaurant": restaurant,
            "Date": date,
            "Time": time,
            "Name": name,
            "Number of People": number_of_people,
            "Status": status
        }
        reservations.append(reservation)
        write_reservations_to_csv(reservations)
        return reservation
    else:
        print("Sorry, the reservation is not available. Please choose a different time or restaurant.")
        return None


def check_reservation_availability(reservations, reservation_id, restaurant, date, time, number_of_people):
    """Checks the availability of a reservation before confirming it."""
    if len(reservations) == 0:
        return True

    conflicting_reservations = [reservation for reservation in reservations
                                if reservation["Restaurant"] == restaurant
                                and reservation["Date"] == date
                                and reservation["Time"] == time
                                and reservation["Reservation ID"] != reservation_id]
    total_people_booked = sum(int(reservation["Number of People"]) for reservation in conflicting_reservations)
    max_capacity = get_max_capacity(restaurant)
    available_capacity = max_capacity - total_people_booked

    if available_capacity >= number_of_people:
        return True
    else:
        return False


def get_max_capacity(restaurant):
    """Returns the maximum capacity for a restaurant."""
    # Assuming the maximum capacity is stored in the restaurants list or in a separate data structure
    max_capacity_dict = {
        "Restaurant A": 30,
        "Restaurant B": 40,
        "Restaurant C": 50
    }
    return max_capacity_dict.get(restaurant, 0)  # Default capacity is 0 if restaurant not found


def read_reservations_from_csv():
    """Reads the reservations from the booking_table.csv file."""
    reservations = []
    try:
        with open(BOOKING_TABLE_FILE, "r", encoding="utf-8-sig") as file:
            reader = csv.DictReader(file)
            for row in reader:
                reservations.append(row)
    except FileNotFoundError:
        pass
    return reservations


def write_reservations_to_csv(reservations):
    """Writes the reservations to the booking_table.csv file."""
    with open(BOOKING_TABLE_FILE, "w", newline="", encoding="utf-8-sig") as file:
        fieldnames = ["Reservation ID", "Restaurant", "Date", "Time", "Name", "Number of People", "Status"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(reservations)


def cancel_reservation(reservation_id):
    """Cancels a reservation."""
    reservations = read_reservations_from_csv()
    for index, reservation in enumerate(reservations):
        if reservation["Reservation ID"] == reservation_id:
            reservations.pop(index)
            write_reservations_to_csv(reservations)
            print("Reservation canceled successfully!")
            return
    print("Reservation not found.")
    return None


def main():
    """Main function."""
    user_credentials = read_user_credentials_from_csv()
    user = None
    while True:
        print("================================================================================")
        print("       😍 The fondest memories are made when gathered around the table 😍       ")
        print("================================================================================")
        print("\nRestaurant Table Reservation Application")
        print("        --------------------------        ")
        print("   ")
        print("1. Register/Login")
        print("2. Make a Reservation")
        print("3. Cancel a Reservation")
        print("4. Exit")
        print("  ")


        choice = input("Enter your choice: ")

        if choice == "1":
            if os.path.isfile(USER_CREDENTIALS_FILE):
                login_choice = input("Do you have an account? (yes/no): ")
                if login_choice.lower() == "yes":
                    user = login(user_credentials)
                    if user:
                        # User is logged in, continue with the rest of the code
                        print("User is logged in.")
                    else:
                        print("Invalid username or password.")
                        return
                elif login_choice.lower() == "no":
                    register_user(user_credentials)
                    user = login(user_credentials)
                    if user:
                        print("User is logged in.")
                    else:
                        print("Invalid username or password.")
                        return
                else:
                    print("Invalid choice. Please enter 'yes' or 'no'.")
                    return
            else:
                print("User credentials file not found.")
                return
        elif choice == "2":
            if user is None:
                print("\nYou need to login first.")
            else:
                print("\nMake a Reservation")
                restaurants = read_restaurants_from_csv()
                restaurant = select_restaurant(restaurants)
                date = input("Enter the date (YYYY-MM-DD): ")
                time = input("Enter the time: ")
                name = input("Enter your name: ")
                number_of_people = input("Enter the number of people: ")
                status = "Confirmed"
                if is_restaurant_in_high_demand(restaurant["name"], date):
                    status = "Pending"
                reservation = create_reservation(restaurant["name"], date, time, name, number_of_people, status)
                if reservation:
                    print("Reservation created successfully!")
        elif choice == "3":
            if user is None:
                print("\nYou need to login first.")
            else:
                reservation_id = input("Enter the reservation ID: ")
                cancel_reservation(reservation_id)
        elif choice == "0":
            print("Thank you for using the Restaurant Table Reservation Application!")
            break
        else:
            print("Invalid choice. Please enter a valid option.")


if __name__ == "__main__":
    main()


       😍 The fondest memories are made when gathered around the table 😍       

Restaurant Table Reservation Application
        --------------------------        
   
1. Register/Login
2. Make a Reservation
3. Cancel a Reservation
4. Exit
  
